# Not-so-simple phi2 chatbot

This notebook uses all the pieces explained in

+ `experiments.ipyn` and
+ `experiments_langchain.ipyn`

to implement a chatbot that can

+ classify input queries into 3 different categories (`support`, `sales` and `joke`),
+ respond accordingly to each of the user queries depending on the category they belong to.

## Putting it all together

Now, in order to put it all together (prompting, conditional branching and conversation memory), we first need to modify a bit our prompts from the _routing_ section. They were meant to be used standalone and they make use of the `Instruction/Output` template from `phi-2`. When we add the conversation bits, this doesn't make sense anymore.

We're going to do this in 3 steps:
1. Create the classification chain
2. Add the branch template
3. Add the conversation memory

## Step 0: load the model

The very first thing we have to do is to load the phi model in a format that can be used with LangChain. We've created some conveniency functions for that in `load_phi_model.py`.

In [1]:
from load_phi_model import load_phi_model_and_tokenizer, get_langchain_model

model, tokenizer = load_phi_model_and_tokenizer()
hf = get_langchain_model(model, tokenizer)

/home/grego/code/jobsearch/ipglobal-2/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Your device is cuda


Loading checkpoint shards: 100%|██████████████████| 2/2 [00:03<00:00,  1.92s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### First, create the initial classification template

We change `text` variable to `human_input` as this is the name we'll use for the chat at the end.

In [2]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableBranch, RunnableLambda, RunnablePassthrough
from operator import itemgetter
from langchain.memory import ConversationBufferMemory

In [3]:
classification_template = """
Instruct: Classify the following text in one of the following categories: ["support", "sales", "joke"]. Output only the name of the category.
+ "support" for customer support texts
+ "sales" for sales and comercial texts
+ "joke" for jokes, funny or comedy like texts
Text: {human_input}
Output:
""".strip()

In [4]:
classification_prompt = ChatPromptTemplate.from_template(classification_template)
classification_chain = (
    classification_prompt
    | hf
    | StrOutputParser()
)

In [5]:
print(classification_chain.invoke({"human_input": "Can I track my order? I'm eager to know its status"}))

 support



### Second, add the branch template

The branch template gives the right instructions for the chatbot.

In [6]:
support_instructions = """\
You are a customer support agent. It seems that the user may have some issues. Answer to their query politely and sincerely. \
Be kind, understanding and say you're sorry for the inconvenience or the situation whenever necessary.\
"""

sales_instructions = """\
Instruction: You are an aggressive salesperson. The user is looking for some information on products. \
Reply to their query by giving information on related products and showcasing how good they are and why they should buy them.\
"""

joke_instructions = """\
Instruction: You are a comedian. The user want's to have some fun. Reply to their query in a funny way.\
"""

general_instructions = """\
Instruction: Respond to the following query.\
"""

support_chain = PromptTemplate.from_template(support_instructions)
sales_chain = PromptTemplate.from_template(sales_instructions)
joke_chain = PromptTemplate.from_template(joke_instructions)
general_chain = PromptTemplate.from_template(general_instructions)

In [7]:
branch = RunnableBranch(
    (lambda x: "support" in x["topic"].lower(), support_chain),
    (lambda x: "sales" in x["topic"].lower(), sales_chain),
    (lambda x: "joke" in x["topic"].lower(), joke_chain),
    general_chain,
) | RunnableLambda(lambda x: x.text)

branch_chain = {"topic": classification_chain, "human_input": lambda x: x["human_input"]} | branch

In [8]:
response = branch_chain.invoke({"human_input": "Can I track my order? I'm eager to know its status"})

In [9]:
response

"You are a customer support agent. It seems that the user may have some issues. Answer to their query politely and sincerely. Be kind, understanding and say you're sorry for the inconvenience or the situation whenever necessary."

### Third and final, add the memory and chat template

In [10]:
template = """\
You are a chatbot having a conversation with a human. Follow the given instructions to reply to the Human message below.

Instructions:{instructions}

{chat_history}
Human: {human_input}
Chatbot:"""

prompt = PromptTemplate(
    input_variables=["instructions", "chat_history", "human_input"], template=template
)
memory = ConversationBufferMemory(
    return_messages=False,
    ai_prefix="Chatbot", 
    human_prefix="Human",
    memory_key="chat_history"
)

In [11]:
def print_prompt(prompt):
    # Add `RunnableLambda(print_prompt)` to the chain to see the prompt at this point
    print("---FULL PROMPT---")
    print(prompt.text)
    print("---END  PROMPT---")
    return prompt

chat_chain = (
    {
        "human_input": lambda x: x["human_input"], 
        "instructions": lambda x: branch_chain,
        "chat_history": (RunnableLambda(memory.load_memory_variables) | itemgetter("chat_history")),
    } | prompt | RunnableLambda(print_prompt) | hf
)

In [12]:
from langchain.callbacks.tracers import ConsoleCallbackHandler

input_ = {"human_input": "Can I track my order? I'm eager to know its status"}

response = chat_chain.invoke(
    input_
    #, config={'callbacks': [ConsoleCallbackHandler()]}
)
memory.save_context({"input": input_['human_input']}, {"ouput": response.strip()})
response

---FULL PROMPT---
You are a chatbot having a conversation with a human. Follow the given instructions to reply to the Human message below.

Instructions:You are a customer support agent. It seems that the user may have some issues. Answer to their query politely and sincerely. Be kind, understanding and say you're sorry for the inconvenience or the situation whenever necessary.


Human: Can I track my order? I'm eager to know its status
Chatbot:
---END  PROMPT---


" I'm sorry to hear that you're eager to know the status of your order. I understand how important it is to you. Could you please provide me with your order number so that I can look into it for you?\n"

## Add user interface

In [15]:
import gradio as gr

In [17]:
HUMAN_NAME = "Human"
BOT_NAME = "Chatbot"

In [18]:
def predict(message, history):
    history_transformer_format = history + [[message, ""]]
    stop_on_tokens = StopOnTokens()
    stop_on_names = StopOnNames(
        [tokenizer.encode(HUMAN_NAME), tokenizer.encode(BOT_NAME)])

    messages = "".join(["".join(
        [f"\n{HUMAN_NAME}: "+item[0], f"\n{BOT_NAME}:"+item[1]]
    ) for item in history_transformer_format]).strip()
    messages = CONTEXT + '\n' + messages

    model_inputs = tokenizer([messages], return_tensors="pt").to(device)
    streamer = TextIteratorStreamer(tokenizer, timeout=10., 
                                    skip_prompt=True, skip_special_tokens=True)
    generate_kwargs = dict(
        model_inputs,
        streamer=streamer,
        max_new_tokens=256,
        do_sample=True,
        top_p=0.95,
        top_k=1000,
        temperature=1.0,
        num_beams=1,
        stopping_criteria=StoppingCriteriaList([stop_on_tokens, stop_on_names])
        )
    t = Thread(target=model.generate, kwargs=generate_kwargs)
    t.start()

    partial_message = ""
    for new_token in streamer:
        partial_message += new_token
        match = re.search(chat_name_pattern_end, partial_message)
        if match:
            partial_message = partial_message[:-len(match.group())]
        yield partial_message


gr.ChatInterface(predict,**DETAILS[LANG]).queue().launch()

NameError: name 'DETAILS' is not defined